In [1]:
cd ..

/home/dmitriishubin/Desktop/Thesis/HepaticVessel_experiment


In [2]:
import pandas as pd
import numpy as np
import pydicom as dicom
import nibabel as nib
import os
from sklearn.model_selection import KFold
import json
import matplotlib.pyplot as plt
import cv2

/home/dmitriishubin/anaconda3/envs/thesis/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [3]:
DATA_PATH = '../data/Task06_Lung/'

In [5]:
img = nib.load(f'{DATA_PATH}imagesTr/lung_003.nii.gz')
img = np.array(img.dataobj)

In [9]:
patients = [i for i in os.listdir(f'{DATA_PATH}imagesTr/') if i.find('._lu')==-1]

In [10]:
patients

['lung_054.nii.gz',
 'lung_047.nii.gz',
 'lung_014.nii.gz',
 'lung_093.nii.gz',
 'lung_042.nii.gz',
 'lung_095.nii.gz',
 'lung_006.nii.gz',
 'lung_037.nii.gz',
 'lung_023.nii.gz',
 'lung_058.nii.gz',
 'lung_055.nii.gz',
 'lung_065.nii.gz',
 'lung_022.nii.gz',
 'lung_066.nii.gz',
 'lung_073.nii.gz',
 'lung_071.nii.gz',
 'lung_031.nii.gz',
 'lung_086.nii.gz',
 'lung_053.nii.gz',
 'lung_025.nii.gz',
 'lung_048.nii.gz',
 'lung_096.nii.gz',
 'lung_003.nii.gz',
 'lung_043.nii.gz',
 'lung_079.nii.gz',
 'lung_045.nii.gz',
 'lung_036.nii.gz',
 'lung_028.nii.gz',
 'lung_016.nii.gz',
 'lung_041.nii.gz',
 'lung_062.nii.gz',
 'lung_070.nii.gz',
 'lung_075.nii.gz',
 'lung_018.nii.gz',
 'lung_059.nii.gz',
 'lung_004.nii.gz',
 'lung_069.nii.gz',
 'lung_057.nii.gz',
 'lung_064.nii.gz',
 'lung_001.nii.gz',
 'lung_015.nii.gz',
 'lung_027.nii.gz',
 'lung_081.nii.gz',
 'lung_005.nii.gz',
 'lung_049.nii.gz',
 'lung_051.nii.gz',
 'lung_061.nii.gz',
 'lung_092.nii.gz',
 'lung_078.nii.gz',
 'lung_083.nii.gz',


In [12]:
names = []

os.makedirs(f'{DATA_PATH}processed_data',exist_ok=True)

count = 0

for patient in patients:
    img = nib.load(f'{DATA_PATH}imagesTr/{patient}')
    img = np.array(img.dataobj)
    seg = nib.load(f'{DATA_PATH}labelsTr/{patient}')
    seg = np.array(seg.dataobj)
    
    patient = patient[:-7]
    
    os.makedirs(f'{DATA_PATH}processed_data/{patient}',exist_ok=True)
    
    for i in range(img.shape[-1]):
        temp_img = img[:,:,i]
        temp_seg = seg[:,:,i]
        
        temp_img = temp_img.reshape(temp_img.shape[0], temp_img.shape[1],1)
        temp_seg = temp_seg.reshape(temp_seg.shape[0], temp_seg.shape[1],1)
        
        
        np.save(f'{DATA_PATH}processed_data/{patient}/{i}.npy',temp_img)
        np.save(f'{DATA_PATH}processed_data/{patient}/{i}_seg.npy',temp_seg)
        count+=1
            

In [13]:
for index,patient in enumerate(patients):
    patients[index]=patient[:-7]

In [14]:
patients = np.array(patients)

kf = KFold(5,shuffle=True,random_state=42)
for fold,(train_index, test_index) in enumerate(kf.split(patients)):
    patients_train, patients_test = patients[train_index], patients[test_index]
    
    split = {
        'train': patients_train.tolist(),
        'val': patients_test.tolist(),
    }
    
    with open(f'../data/split_table/{fold}_split_table.json', 'w') as outfile:
            json.dump(split, outfile)